# Conférences Python Master TIDE #3

## A) Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :
- https://webstat.banque-france.fr/fr/#/node/5385566

Les données sont dans le fichier `Webstat_Export.csv`.

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

1) Charger le fichier dans un DataFrame dont l'index doit correspondre aux dates, triées dans l'ordre chronologique et les colonnes doivent être les codes ISO3 des devises.

2) Sélectionner les devises : ['USD', 'CHF', 'GBP', 'JPY', 'TRY', 'CNY']

3) Calculer la moyenne des taux de change par année. Voir les codes de resampling : https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects

4) Faire des graphiques avec :
- Les taux de change divisés par leur moyenne.
- Les taux de change divisés par leur moyenne avec une moyenne glissante de 30 jours ou bien un maximum glissant de 100 jours.
- Les taux de change divisés par la dernière valeur connue avec une moyenne glissante de 30 jours.

5) Réindexer le DataFrame en prenant toutes les dates et compléter les valeurs manquantes par interpolation linéaire.

6) Avec la librairie **sparklines** afficher un DataFrame avec la moyenne annuelle des taux de change pour une monnaie donnée ainsi que la tendance par trimestre.

```python
import sparklines

print(sparklines.sparklines(pd.Series(range(8)))[0])
```

## B) Données télécom

Dataset des opérateurs télécom : `couverture-2g-3g-4g-en-france-par-operateur-juillet-2015.csv`

1) Chargez le fichier dans un DataFrame.

2) Transformez ce DataFrame large en un DataFrame long en isolant les colonnes d'identification d'une part et les colonnes de valeurs de couverture d'autre part.

3) Rajoutez 3 colonnes "Opérateur", "Type Couverture" et "Réseau" calculées à partir de la colonne "variable" obtenue en 2.

4) Chargez le DataFrame des communes déjà utilisé et effectuez une jointure sur le code INSEE.

6) Pour le réseau 4G et la couverture surfacique, calculez la moyenne des couvertures par statut de commune et par opérateur. Pour quel couple (statut, opérateur) a-t-on la moyenne de la couverture la plus faible ?

## C) Données de formations

Dataset : `ListeOF.zip`

1) Charger le fichier dans un DataFrame.

2) A partir du nombre de valeurs non nulles des colonnes, styliser un DataFrame avec une barre de couleur.

3) A partir du pourcentage de remplissage des colonnes arrondi à un chiffre, styliser un DataFrame avec une colormap.

&copy; Francis Wolinski 2025

In [2]:
import numpy as np
import pandas as pd

In [20]:
df = pd.read_csv("./data/Webstat_Export.csv",
                 sep=";",
                 header=2,
                 na_values=["", "-"],
                 keep_default_na=False,
                 decimal=",",
                 parse_dates=[0],
                 date_format="%d/%m/%Y")

cols = [col for col in df.columns if (col == df.columns[0]) or (col.startswith("EXR.D.") and col.endswith(".EUR.SP00.A"))]

df = df[cols]

dico_cols = {col:col[6:9] for col in df.columns if col.startswith("EXR.D.") and col.endswith(".EUR.SP00.A")}
dico_cols[df.columns[0]] = "Date"

df = df.rename(columns=dico_cols)

df = df.set_index("Date")

df = df.sort_index()

df

,AUD,BGN,BRL,CAD,CHF,CNY,CYP,CZK,DKK,EEK,...,RON,RUB,SEK,SGD,SIT,SKK,THB,TRY,USD,ZAR
Date,,,,,,,,,,,,,,,,,,,,,
1999-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-04,1.9100,NaN,NaN,1.8004,1.6168,NaN,0.58231,35.107,7.4501,15.6466,...,1.3111,25.2875,9.4696,1.9554,189.045,42.991,42.6799,0.3723,1.1789,6.9358
1999-01-05,1.8944,NaN,NaN,1.7965,1.6123,NaN,0.58230,34.917,7.4495,15.6466,...,1.3168,26.5876,9.4025,1.9655,188.775,42.848,42.5048,0.3728,1.1790,6.7975
1999-01-06,1.8820,NaN,NaN,1.7711,1.6116,NaN,0.58200,34.850,7.4452,15.6466,...,1.3168,27.4315,9.3050,1.9699,188.700,42.778,42.6949,0.3722,1.1743,6.7307
1999-01-07,1.8474,NaN,NaN,1.7602,1.6165,NaN,0.58187,34.886,7.4431,15.6466,...,1.3092,26.9876,9.1800,1.9436,188.800,42.765,42.1678,0.3701,1.1632,6.8283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-13,1.6305,1.9558,5.5486,1.4757,0.9784,7.8095,NaN,24.779,7.4608,NaN,...,4.9762,NaN,11.7000,1.4605,NaN,NaN,39.6880,34.7702,1.0795,19.8185


In [17]:
cols = [col for col in df.columns if (col == df.columns[0]) or (col.startswith("EXR.D.") and col.endswith(".EUR.SP00.A"))]
cols

['Code série :',
 'EXR.D.AUD.EUR.SP00.A',
 'EXR.D.BGN.EUR.SP00.A',
 'EXR.D.BRL.EUR.SP00.A',
 'EXR.D.CAD.EUR.SP00.A',
 'EXR.D.CHF.EUR.SP00.A',
 'EXR.D.CNY.EUR.SP00.A',
 'EXR.D.CYP.EUR.SP00.A',
 'EXR.D.CZK.EUR.SP00.A',
 'EXR.D.DKK.EUR.SP00.A',
 'EXR.D.EEK.EUR.SP00.A',
 'EXR.D.GBP.EUR.SP00.A',
 'EXR.D.HKD.EUR.SP00.A',
 'EXR.D.HRK.EUR.SP00.A',
 'EXR.D.HUF.EUR.SP00.A',
 'EXR.D.IDR.EUR.SP00.A',
 'EXR.D.ILS.EUR.SP00.A',
 'EXR.D.INR.EUR.SP00.A',
 'EXR.D.ISK.EUR.SP00.A',
 'EXR.D.JPY.EUR.SP00.A',
 'EXR.D.KRW.EUR.SP00.A',
 'EXR.D.LTL.EUR.SP00.A',
 'EXR.D.LVL.EUR.SP00.A',
 'EXR.D.MTL.EUR.SP00.A',
 'EXR.D.MXN.EUR.SP00.A',
 'EXR.D.MYR.EUR.SP00.A',
 'EXR.D.NOK.EUR.SP00.A',
 'EXR.D.NZD.EUR.SP00.A',
 'EXR.D.PHP.EUR.SP00.A',
 'EXR.D.PLN.EUR.SP00.A',
 'EXR.D.RON.EUR.SP00.A',
 'EXR.D.RUB.EUR.SP00.A',
 'EXR.D.SEK.EUR.SP00.A',
 'EXR.D.SGD.EUR.SP00.A',
 'EXR.D.SIT.EUR.SP00.A',
 'EXR.D.SKK.EUR.SP00.A',
 'EXR.D.THB.EUR.SP00.A',
 'EXR.D.TRY.EUR.SP00.A',
 'EXR.D.USD.EUR.SP00.A',
 'EXR.D.ZAR.EUR.SP00.A']

In [16]:
dico_cols = {col:col[6:9] for col in df.columns if col.startswith("EXR.D.") and col.endswith(".EUR.SP00.A")}
dico_cols[df.columns[0]] = "Date"
dico_cols

{'EXR.D.AUD.EUR.SP00.A': 'AUD',
 'EXR.D.BGN.EUR.SP00.A': 'BGN',
 'EXR.D.BRL.EUR.SP00.A': 'BRL',
 'EXR.D.CAD.EUR.SP00.A': 'CAD',
 'EXR.D.CHF.EUR.SP00.A': 'CHF',
 'EXR.D.CNY.EUR.SP00.A': 'CNY',
 'EXR.D.CYP.EUR.SP00.A': 'CYP',
 'EXR.D.CZK.EUR.SP00.A': 'CZK',
 'EXR.D.DKK.EUR.SP00.A': 'DKK',
 'EXR.D.EEK.EUR.SP00.A': 'EEK',
 'EXR.D.GBP.EUR.SP00.A': 'GBP',
 'EXR.D.HKD.EUR.SP00.A': 'HKD',
 'EXR.D.HRK.EUR.SP00.A': 'HRK',
 'EXR.D.HUF.EUR.SP00.A': 'HUF',
 'EXR.D.IDR.EUR.SP00.A': 'IDR',
 'EXR.D.ILS.EUR.SP00.A': 'ILS',
 'EXR.D.INR.EUR.SP00.A': 'INR',
 'EXR.D.ISK.EUR.SP00.A': 'ISK',
 'EXR.D.JPY.EUR.SP00.A': 'JPY',
 'EXR.D.KRW.EUR.SP00.A': 'KRW',
 'EXR.D.LTL.EUR.SP00.A': 'LTL',
 'EXR.D.LVL.EUR.SP00.A': 'LVL',
 'EXR.D.MTL.EUR.SP00.A': 'MTL',
 'EXR.D.MXN.EUR.SP00.A': 'MXN',
 'EXR.D.MYR.EUR.SP00.A': 'MYR',
 'EXR.D.NOK.EUR.SP00.A': 'NOK',
 'EXR.D.NZD.EUR.SP00.A': 'NZD',
 'EXR.D.PHP.EUR.SP00.A': 'PHP',
 'EXR.D.PLN.EUR.SP00.A': 'PLN',
 'EXR.D.RON.EUR.SP00.A': 'RON',
 'EXR.D.RUB.EUR.SP00.A': 'RUB',
 'EXR.D.